# Lesson 05 Assignement
# Houda Aynaou

## Workplace Scenario

Rooney's client is a tech-manufacturing startup working on a number of automated detection devices for the medical and construction industries. Among the auto-detection devices is a reader that looks at possible carcinoma tissue samples to classify the sample as either benign or malignant. Rooney asks you for help in developing a better algorithm than the current classifier, perhaps a decision tree can help.

For this assignment, you will be designing an experiment using decision tree classifiers for the detection of breast cancer and comparing the accuracy using [Breast Cancer Wisconsin Data Set](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)).

| Column                    |Description      |
|---------------------------|-----------------|
|Sample code number         | id number       |
|Class                      | 4 for malignant, 2 for benign |
|Clump Thickness            | 1 - 10 |     
|Uniformity of Cell Size    | 1 - 10 |
|Uniformity of Cell Shape   | 1 - 10 |
|Marginal Adhesion          | 1 - 10 |
|Single Epithelial Cell Size| 1 - 10 |
|Bare Nuclei                | 1 - 10 |
|Normal Nucleoli            | 1 - 10 |
|Mitosis                    | 1 - 10 |



## To do

1. Test both entropy and the gini coefficient. Which performs better and why? **give evidence with one or more validation measures**
2. What are the best hyperparameter settings for both? **GridSearchCV()**
3. Visualize both models and see which feature is selected for each criterion. Are they same for both? Why or why not? **graphviz**
4. Determine the AUC for the best model you can achieve. What are the precision and recal values and which might be the one you want to maximize?
5. What are the implications of using this type of machine learning algorithm for breast cancer analysis? **Is it a good thing? Is the performance good enough?**

In [14]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier


## 1. Data

In [24]:
LINK = 'https://raw.githubusercontent.com/houdaaynaou/DS-Certificate-UW/master/Course%203%20Machine%20Learning%20Techniques/Data/WI_Breast_Cancer.csv'
col_names = ['ID', 'Class', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitosis' ]

data = pd.read_csv(LINK, names= col_names)
data.head()


,ID,Class,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitosis
0,1000025,2,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1
1,1002945,2,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1
2,1015425,2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1
3,1016277,2,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1
4,1017023,2,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1


In [25]:
data.shape

(709, 11)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 11 columns):
ID                             709 non-null object
Class                          702 non-null object
Clump Thickness                702 non-null float64
Uniformity of Cell Size        701 non-null float64
Uniformity of Cell Shape       701 non-null float64
Marginal Adhesion              701 non-null float64
Single Epithelial Cell Size    701 non-null float64
Bare Nuclei                    701 non-null float64
Bland Chromatin                701 non-null float64
Normal Nucleoli                701 non-null float64
Mitosis                        701 non-null object
dtypes: float64(8), object(3)
memory usage: 61.0+ KB


In [27]:
data.describe()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli
count,702.000000,701.000000,701.00000,701.000000,701.000000,701.000000,701.000000,701.000000
mean,7.263533,3.154066,3.21826,2.813124,3.226819,3.456491,3.447932,2.877318
std,75.031332,3.069053,2.97456,2.858201,2.220340,3.637881,2.447550,3.061150
min,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,2.000000,1.000000,1.00000,1.000000,2.000000,1.000000,2.000000,1.000000
50%,4.000000,1.000000,1.00000,1.000000,2.000000,1.000000,3.000000,1.000000
75%,6.000000,5.000000,5.00000,4.000000,4.000000,5.000000,5.000000,4.000000
max,1991.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000


In [37]:
data[data['Mitosis'] == '3) <-----Reappears in Group 8 as:']

,ID,Class,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitosis
539,##### 1268952,4,10.0,10.0,7.0,2.0,7.0,1.0,4.0,3.0,3) <-----Reappears in Group 8 as:


In [38]:
data.Class.unique()

array(['2', '4', nan, ' 1990; Updated Jan 8'], dtype=object)